## Na co bacha

- Některé pohledy nutno filtrovat podle "mluvci_jisty", jinak stránky jako *2002ps_stenprot_001schuz_s001012.htm* dělají ze Zaorálka ženu. Ale ne všechny pohledy, pokud chceme vědět, kdo toho napovídal nejvíc – takové projevy se ženou přes víc stránek.
- Filtrovat procedurální věci podle pro návrh / proti návrhu

In [252]:
import os
import polars as pl

In [253]:
from src.najdi_tituly import najdi_tituly

In [254]:
import altair as alt

In [255]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [256]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1993
).filter(
    pl.col("komora_komplet").str.contains('Poslanecká') | pl.col('komora_komplet').str.contains('PČR, PS')
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).collect()

In [257]:
alt.Chart(
    df.group_by_dynamic(index_column="datum",every="1m").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [258]:
alt.Chart(
    df.group_by_dynamic(index_column="datum",every="1m").agg(pl.col("mluvci").unique().len()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("mluvci:Q")
)

alt.Chart(...)

In [259]:
alt.Chart(
    df.group_by_dynamic(index_column="datum",every="1m").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [260]:
df.filter(pl.col('mluvci_id').is_null()).group_by("obdobi").agg(pl.col('pocet_slov').sum())

obdobi,pocet_slov
i32,u32
1993,1004559


In [261]:
df.filter(pl.col('soubor') == '1996ps_stenprot_016schuz_s016002.htm')

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Poslanec Miroslav Sládek""","""1996ps_stenprot_016schuz_s016002.htm""","""/sqw/detail.sqw?id=48&o=2""","""Pane předsedající, dámy a pánové, abych nemusel posléze přerušovat řeč, tak bych rád prostřednictvím pana předsedajícího upozornil poslance, že nemusí…",1,"""ps""",1996,16,2,1997-11-11,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""16. schůze""",true,true,true,"""48&o=2""",2037


In [262]:
df.group_by('obdobi').agg(pl.col('text').unique().len()).sort(by='obdobi')

obdobi,text
i32,u32
1993,29829
1996,30222
1998,77330
2002,78852
2006,55577
2010,46019
2013,82396
2017,89064
2021,82042


In [263]:
df.filter(pl.col("id").is_null())

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Předseda PSP Milan Uhde""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Vážené paní poslankyně, vážení páni poslanci, vážené dámy a pánové, zahajuji slavnostní schůzi Poslanecké sněmovny Parlamentu České republiky a všechn…",3,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,175
"""Místopředseda PSP Jiří Vlach""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Vážené dámy a pánové, přistoupíme k prvému bodu pořadu naší schůze, a prosím, aby se slova ujal předseda Poslanecké sněmovny pan Milan Uhde.""",4,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,22
"""Předseda PSP Milan Uhde""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Vážený pane předsedo vlády, vážení členové vlády, vážení členové diplomatického sboru, vážené paní poslankyně, vážení páni poslanci, vážení představit…",5,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,1075
"""Místopředseda PSP Jiří Vlach""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Děkuji předsedovi Poslanecké sněmovny panu Milanovi Uhdemu a nyní prosím, aby se ujal slova předseda vlády České republiky pan Václav Klaus.""",6,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,20
"""Předseda vlády ČR Václav Klaus""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Vážené paní poslankyně, vážení páni poslanci, dámy a pánové, jistě nemusím zdůrazňovat, že jsem si vědom toho, že před vás předstupuji ve chvíli, kter…",7,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,1852
"""Místopředseda PSP Jiří Vlach""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Děkuji předsedovi vlády panu Václavu Klausovi. Nyní projednáme třetí bod našeho programu, kterým je Prosím předsedu Poslanecké sněmovny pana Milana U…",8,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,26
"""Předseda PSP Milan Uhde""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Prohlášení České národní rady k parlamentům a národům světa Česká národní rada, jako svobodně zvolený zákonodárný sbor České republiky, pamětliva tisí…",9,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,510
"""Místopředseda PSP Jiří Vlach""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Děkuji předsedovi Poslanecké sněmovny panu Milanu Uhdemu. Vážené dámy a pánové, dovolte mi, abych vám před závěrem naší slavnostní schůze sdělil ještě…",10,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,124
"""Předseda PSP Milan Uhde""","""1993ps_stenprot_002schuz_s002001.htm""",null,"""Vážené paní poslankyně, vážení páni poslanci, milí hosté, zjišťuji, že podle prezenční listiny bylo v 11.02 hod. podepsáno, a tudíž i přítomno 127 pos…",3,"""ps""",1993,2,1,1993-01-26,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""2. schůze""",true,true,true,null,559


In [264]:
df.sample(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Poslanec Tomio Okamura""","""2021ps_stenprot_033schuz_s033109.htm""","""/sqw/detail.sqw?id=6105""","""Je to důležitá věc, že vy berete tomu zdravotnímu systému 14 miliard korun a zhoršujete dostupnost zdravotní péče pro naše občany a pracovní podmínky …",1,"""ps""",2021,33,109,2022-08-27,"""PČR, PS 2021-...""","""33. schůze""",true,true,false,"""6105""",137
"""Místopředsedkyně PSP Jaroslava Jermanová""","""2013ps_stenprot_036schuz_s036331.htm""","""/sqw/detail.sqw?id=6202""","""Děkuji a prosím k mikrofonu pana poslance Paveru, připraví se pan poslanec Kučera. Všechno jsou faktické poznámky. Prosím, máte slovo.""",4,"""ps""",2013,36,331,2015-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""36. schůze""",true,true,true,"""6202""",19
"""Místopředseda vlády a ministr financí ČR Pavel Mertlík""","""1998ps_stenprot_027schuz_s027145.htm""","""http://www.vlada.cz/1250/vlada/vlada_clenove.htm""","""Navrhovatel rovněž doporučuje.""",17,"""ps""",1998,27,145,2000-09-20,"""Parlament České republiky, Poslanecká sněmovna 1998 – 2002""","""27. schůze""",true,true,true,null,2
"""Místopředsedkyně PSP Kateřina Klasnová""","""2010ps_stenprot_005schuz_s005112.htm""","""/sqw/detail.sqw?id=5935""","""Dobrý den. Dámy a pánové, já tady chci tlumočit především stanovisko Ministerstva pro místní rozvoj, protože pan ministr Jankovský je z dnešní schůze …",3,"""ps""",2010,5,112,2010-09-24,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""5. schůze""",true,true,true,"""5935""",357
"""Poslanec Stanislav Grospič""","""2010ps_stenprot_036schuz_s036041.htm""","""/sqw/detail.sqw?id=4778""","""Já bych chtěl říci, kolegyně, kolegové, že vítám tu diskusi z pravého spektra Poslanecké sněmovny k limitům volební kampaně prezidenta, a chtěl bych t…",5,"""ps""",2010,36,41,2012-03-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""36. schůze""",true,true,true,"""4778""",224
"""Předseda PSP Václav Klaus""","""1998ps_stenprot_043schuz_s043116.htm""","""/sqw/p.sqw?P=493""","""Ano, procedurální návrh.""",12,"""ps""",1998,43,116,2001-11-29,"""Parlament České republiky, Poslanecká sněmovna 1998 – 2002""","""43. schůze""",true,true,true,null,2
"""Předsedkyně PSP Miroslava Němcová""","""2010ps_stenprot_003schuz_s003013.htm""","""/sqw/detail.sqw?id=401""","""Čas, pane kolego!""",6,"""ps""",2010,3,13,2010-08-10,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""3. schůze""",true,true,true,"""401""",2
"""Poslanec Pavel Suchánek""","""2010ps_stenprot_009schuz_s009147.htm""","""/sqw/detail.sqw?id=415""","""Děkuji za slovo, já v podstatě jen zopakuji, co jste říkal vy, že rozpočtový výbor dne 9. prosince projednal tento tisk a doporučil Poslanecké sněmovn…",4,"""ps""",2010,9,147,2010-12-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""9. schůze""",true,true,true,"""415""",29
"""Poslanec Jiří Carbol""","""2006ps_stenprot_052schuz_s052165.htm""","""/sqw/detail.sqw?id=5449""","""Děkuji za slovo, paní předsedající. Vážené kolegyně a vážení kolegové, já bych navázal na slova předpředřečníka, který hovořil o některých těch případ…",6,"""ps""",2006,52,165,2009-03-26,"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""","""52. schůze""",true,true,true,"""5449""",134


In [265]:
df_mluvci = pl.read_parquet(os.path.join("data","mluvci.parquet"))

In [266]:
df_mluvci.filter(pl.col('id').is_null())

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]


In [267]:
df_mluvci.sample(10)

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""6760""",2021,"""6760_9.html""","""Ing. Radim Holiš""","""Zlínský""","""m""",2021-10-09,2021-11-10,"""ANO2011""",null,"""12. 8. 1975""","[{""člen"",""Poslanecký klub ANO 2011"",""12. 10. 2021"",""10. 11. 2021""}]",1975,1975-08-12,"""Radim Holiš""","[""Ing.""]"
"""215""",2002,"""215_4.html""","""Ing. Josef Hojdar""","""Ústecký""","""m""",2002-06-15,2006-02-28,"""ČSSD""",null,"""1946""","[{""člen"",""Organizační výbor"",""16. 7. 2002"",""28. 2. 2006""}, {""člen"",""Hospodářský výbor"",""16. 7. 2002"",""28. 2. 2006""}, … {""člen"",""Dozorčí rada České konsolidační agentury"",""10. 7. 2001"",""10. 7. 2006""}]",1946,null,"""Josef Hojdar""","[""Ing.""]"
"""6479""",2021,"""6479_9.html""","""Bc. Lucie Šafránková""","""Jihomoravský""","""f""",2021-10-09,2021-11-23,"""SPD""",null,"""21. 5. 1987""","[{""člen"",""Výbor pro sociální politiku"",""23. 11. 2021"",null}, {""místopředsed"",""Výbor pro sociální politiku"",""24. 11. 2021"",null}, … {""člen"",""Skupina ČR - Indie"",""1. 2. 2023"",null}]",1987,1987-05-21,"""Lucie Šafránková""","[""Bc.""]"
"""6184""",2017,"""6184_8.html""","""Mgr. Richard Brabec""","""Ústecký""","""m""",2017-10-21,2021-10-20,"""ANO2011""",null,"""1966""","[{""člen"",""Poslanecký klub ANO 2011"",""24. 10. 2017"",""20. 10. 2021""}, {null,""Vláda České republiky"",""29. 1. 2014"",""13. 12. 2017""}, … {null,""Vláda České republiky"",""27. 6. 2018"",""17. 12. 2021""}]",1966,null,"""Richard Brabec""","[""Mgr.""]"
"""282""",1998,"""282_3.html""","""Mgr. Oldřich Vojíř, Ph.D.""","""Severočeský""","""m""",1998-06-20,2002-06-20,"""ODS""",null,"""1961""","[{""člen"",""Hospodářský výbor"",""22. 7. 1998"",""20. 6. 2002""}, {null,""Hospodářský výbor"",""23. 7. 1998"",""20. 6. 2002""}, … {""člen"",""Dozorčí rada Fondu národního majetku České republiky"",""2. 4. 1999"",""9. 10. 2002""}]",1961,null,"""Oldřich Vojíř""","[""Mgr."", ""Ph.D.""]"
"""6773""",2021,"""6773_9.html""","""Mgr. Zdeněk Kettner""","""Ústecký""","""m""",2021-10-09,2021-11-10,"""SPD""",null,"""4. 2. 1974""","[{""člen"",""Výbor pro vědu, vzdělání, kulturu, mládež a tělovýchovu"",""23. 11. 2021"",null}, {""místopředsed"",""Výbor pro vědu, vzdělání, kulturu, mládež a tělovýchovu"",""24. 11. 2021"",null}, … {""člen"",""Skupina ČR - Arménie"",""9. 6. 2022"",null}]",1974,1974-02-04,"""Zdeněk Kettner""","[""Mgr.""]"
"""6504""",2017,"""6504_8.html""","""Jiří Kohoutek""","""Pardubický""","""m""",2017-10-21,2021-10-20,"""SPD""",null,"""1964""","[{""člen"",""Hospodářský výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Výbor pro veřejnou správu a regionální rozvoj"",""28. 11. 2017"",""20. 10. 2021""}, … {""člen"",""Skupina ČR - Gruzie"",""30. 5. 2019"",""20. 10. 2021""}]",1964,null,"""Jiří Kohoutek""",[]
"""5433""",2010,"""5433_6.html""","""MUDr. Jiří Koskuba""","""Praha""","""m""",2010-05-29,2013-08-28,"""ČSSD""",null,"""1955""","[{""člen"",""Výbor pro zdravotnictví"",""7. 7. 2010"",""28. 8. 2013""}, {""člen"",""Poslanecký klub České strany sociálně demokratické"",""4. 6. 2010"",""28. 8. 2013""}, {""člen"",""Správní rada VZP"",""22. 9. 2010"",""6. 2. 2014""}]",1955,null,"""Jiří Koskuba""","[""MUDr.""]"
"""241""",1996,"""241_2.html""","""PhDr. Vladimír Špidla""","""Jihočeský""","""m""",1996-06-01,1998-06-19,"""ČSSD""",null,"""1951""","[{""člen"",""Výbor pro sociální politiku a zdravotnictví"",""2. 7. 1996"",""19. 6. 1998""}, {""předsed"",""Výbor pro sociální politiku a zdravotnictví"",""3. 7. 1996"",""19. 6. 1998""}, {""člen"",""Poslanecký klub České strany sociálně demokratické"",""12. 6. 1996"",""19. 6. 1998""}]",1951,null,"""Vladimír Špidla""","[""PhDr.""]"


In [268]:
df_mluvci.sort(
    by=['id','jmeno']
).unique(
    subset=['id','jmeno'], keep='first'
)

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""392""",1998,"""392_3.html""","""Ing. Jaroslav Lobkowicz""","""Západočeský""","""m""",1998-06-20,2002-06-20,"""KDU-ČSL""",null,"""1942""","[{""člen"",""Zemědělský výbor"",""22. 7. 1998"",""20. 6. 2002""}, {""člen"",""Výbor pro evropskou integraci"",""22. 7. 1998"",""20. 6. 2002""}, … {""člen"",""Poslanecký klub Křesťanské a demokratické unie - Československé strany lidové"",""25. 6. 1998"",""20. 6. 2002""}]",1942,null,"""Jaroslav Lobkowicz""","[""Ing.""]"
"""5304""",2002,"""5304_4.html""","""RSDr. Antonín Zralý""","""Jihomoravský""","""m""",2002-06-15,2006-06-15,"""KSČM""",null,"""1944""","[{""člen"",""Výbor pro sociální politiku a zdravotnictví"",""16. 7. 2002"",""15. 6. 2006""}, {null,""Výbor pro sociální politiku a zdravotnictví"",""20. 10. 2002"",""15. 6. 2006""}, … {""člen"",""Skupina ČR - Střední Amerika a Karibik"",""29. 4. 2003"",""15. 6. 2006""}]",1944,null,"""Antonín Zralý""","[""RSDr.""]"
"""6471""",2021,"""6471_9.html""","""Mgr. Jana Mračková Vildumetzová""","""Karlovarský""","""f""",2021-10-09,2024-09-21,"""ANO2011""",null,"""8. 5. 1973""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",""30. 9. 2022""}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",""30. 9. 2022""}, … {""místopředsed"",""Poslanecký klub ANO 2011"",""12. 10. 2022"",""21. 9. 2024""}]",1973,1973-05-08,"""Jana Mračková Vildumetzová""","[""Mgr.""]"
"""6735""",2021,"""6735_9.html""","""Jan Kuchař""","""Karlovarský""","""m""",2021-10-09,null,"""PirSTAN""","""STAN""","""18. 5. 1974""","[{""člen"",""Výbor pro zdravotnictví"",""23. 11. 2021"",null}, {""člen"",""Rozpočtový výbor"",""23. 11. 2021"",null}, … {""člen"",""Dozorčí rada VZP"",""18. 2. 2022"",null}]",1974,1974-05-18,"""Jan Kuchař""",[]
"""5451""",2006,"""5451_5.html""","""Ing. Josef Čerňanský""","""Ústecký""","""m""",2006-06-03,2010-06-03,"""ČSSD""",null,"""1943""","[{""člen"",""Zemědělský výbor"",""12. 9. 2006"",""3. 6. 2010""}, {""člen"",""Kontrolní výbor"",""8. 12. 2006"",""3. 6. 2010""}, … {""člen"",""Dozorčí rada Pozemkového fondu České republiky"",""3. 2. 2010"",""1. 1. 2013""}]",1943,null,"""Josef Čerňanský""","[""Ing.""]"
"""5896""",2010,"""5896_6.html""","""MUDr. Jiří Besser""","""Středočeský""","""m""",2010-05-29,2013-08-28,"""TOP09""",null,"""1957""","[{""člen"",""Výbor pro vědu, vzdělání, kulturu, mládež a tělovýchovu"",""15. 12. 2011"",""28. 8. 2013""}, {""člen"",""Volební výbor"",""15. 12. 2011"",""28. 8. 2013""}, … {null,""Vláda České republiky"",""13. 7. 2010"",""16. 12. 2011""}]",1957,null,"""Jiří Besser""","[""MUDr.""]"
"""6768""",2021,"""6768_9.html""","""Ing. et Ing. Miloš Nový""","""Plzeňský""","""m""",2021-10-09,null,"""SPOLU""","""TOP09""","""18. 1. 1960""","[{""člen"",""Rozpočtový výbor"",""23. 11. 2021"",null}, {""člen"",""Výbor pro životní prostředí"",""23. 11. 2021"",null}, … {""člen"",""Skupina ČR - Itálie"",""20. 5. 2022"",null}]",1960,1960-01-18,"""Miloš Nový""","[""Ing."", ""Ing.""]"
"""6773""",2021,"""6773_9.html""","""Mgr. Zdeněk Kettner""","""Ústecký""","""m""",2021-10-09,2021-11-10,"""SPD""",null,"""4. 2. 1974""","[{""člen"",""Výbor pro vědu, vzdělání, kulturu, mládež a tělovýchovu"",""23. 11. 2021"",null}, {""místopředsed"",""Výbor pro vědu, vzdělání, kulturu, mládež a tělovýchovu"",""24. 11. 2021"",null}, … {""člen"",""Skupina ČR - Arménie"",""9. 6. 2022"",null}]",1974,1974-02-04,"""Zdeněk Kettner""","[""Mgr.""]"
"""5933""",2013,"""5933_7.html""","""David Kádner""","""Ústecký""","""m""",2013-10-26,2017-10-26,"""Usvit""",null,"""1976""","[{""člen"",""Výbor pro obranu"",""4. 12. 2013"",""26. 10. 2017""}, {""člen"",""Výbor pro bezpečnost"",""4. 12. 2013"",""12. 12. 2013""}, … {""člen"",""Skupina ČR - Estonsko"",""20. 6. 2014"",""26. 10. 2017""}]",1976,null,"""David Kádner""",[]


In [269]:
df_mluvci.filter(pl.col("jmeno").str.contains("Andrej Babiš"))

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""6150""",2013,"""6150_7.html""","""Ing. Andrej Babiš""","""Praha""","""m""",2013-10-26,2017-10-26,"""ANO2011""",null,"""1954""","[{""člen"",""Rozpočtový výbor"",""4. 12. 2013"",""29. 1. 2014""}, {""člen"",""Kontrolní výbor"",""7. 6. 2017"",""26. 10. 2017""}, … {null,""Vláda České republiky"",""29. 1. 2014"",""24. 5. 2017""}]",1954,null,"""Andrej Babiš""","[""Ing.""]"
"""6150""",2017,"""6150_8.html""","""Ing. Andrej Babiš""","""Středočeský""","""m""",2017-10-21,2021-10-20,"""ANO2011""",null,"""1954""","[{""člen"",""Poslanecký klub ANO 2011"",""24. 10. 2017"",""20. 10. 2021""}, {""předsed"",""Vláda České republiky"",""6. 12. 2017"",""27. 6. 2018""}, {""předsed"",""Vláda České republiky"",""27. 6. 2018"",""17. 12. 2021""}]",1954,null,"""Andrej Babiš""","[""Ing.""]"
"""6150""",2021,"""6150_9.html""","""Ing. Andrej Babiš""","""Ústecký""","""m""",2021-10-09,null,"""ANO2011""",null,"""2. 9. 1954""","[{""člen"",""Poslanecký klub ANO 2011"",""12. 10. 2021"",null}, {""předsed"",""Vláda České republiky"",""27. 6. 2018"",""17. 12. 2021""}]",1954,1954-09-02,"""Andrej Babiš""","[""Ing.""]"


In [270]:
df_mluvci.filter(pl.col("jmeno").str.contains("Tomio Okamura"))

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""6105""",2013,"""6105_7.html""","""Tomio Okamura""","""Středočeský""","""m""",2013-10-26,2017-10-26,"""Usvit""",null,"""1972""","[{""člen"",""Organizační výbor"",""26. 3. 2014"",""30. 4. 2015""}, {""člen"",""Podvýbor pro Ústavu České republiky a parlamentní procedury"",""18. 3. 2014"",""26. 10. 2017""}, … {""člen"",""Skupina ČR - Japonsko"",""13. 2. 2014"",""26. 10. 2017""}]",1972,null,"""Tomio Okamura""",[]
"""6105""",2017,"""6105_8.html""","""Tomio Okamura""","""Středočeský""","""m""",2017-10-21,2021-10-20,"""SPD""",null,"""1972""","[{""člen"",""Organizační výbor"",""24. 11. 2017"",""20. 10. 2021""}, {""místopředsed"",""Organizační výbor"",""24. 11. 2017"",""20. 10. 2021""}, … {""předsed"",""Skupina ČR - Afghánská islámská republika"",""9. 3. 2020"",""20. 10. 2021""}]",1972,null,"""Tomio Okamura""",[]
"""6105""",2021,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]


In [271]:
df.sort(by='datum').filter(pl.col('mluvci').str.contains('Petr Fiala'))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_038schuz_s038048.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní předsedkyně, dámy a pánové, Mezinárodní úmluva proti dopingu ve sportu patří do kategorie smluv, jejichž přijetí nebo přijetí jejich změn …",6,"""ps""",2010,38,48,2012-05-03,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""38. schůze""",true,true,true,null,222
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040146.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní předsedající, dámy a pánové, dovolte mi seznámit vás ve stručnosti s návrhem novely zákona o výkonu ústavní výchovy a ochranné výchovy ve …",10,"""ps""",2010,40,146,2012-06-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,475
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040147.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Ano, paní předsedající, já souhlasím s tímto návrhem na zkrácení lhůty na 48 hodin.""",8,"""ps""",2010,40,147,2012-06-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,13
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040244.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní poslankyně, dámy a pánové, mám tu čest stát před vámi při interpelacích poprvé, tak mi dovolte na úvod jenom krátkou poznámku. Když jsem p…",5,"""ps""",2010,40,244,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,531
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040244.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Ano, děkuji, pane předsedající, nyní odpovím velmi stručně. Jsem si plně vědom problematické situace v oblasti financování regionálního školství. Jsem…",9,"""ps""",2010,40,244,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,110
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040245.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážený pane poslanče, dámy a pánové, děkuji za váš dotaz, který nás přivádí k jiné části širokého resortu, který mám tu čest spravovat, totiž k oblast…",6,"""ps""",2010,40,245,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,323
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040246.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Paní Hana Zejdová se sice v roce 1993 umístila na prvním místě na mistrovství Evropy v bezmotorovém létání, v následujících letech už nebyla tak úspěš…",1,"""ps""",2010,40,246,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,false,null,145
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040246.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""To vysvětlení, pane poslanče, které jsem poskytl, v žádném případě nezpochybňuje jedinečnost výkonů paní Zejdové ani to její momentální úsilí. Podstat…",5,"""ps""",2010,40,246,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,

In [272]:
df.filter(pl.col("id") == "6074")

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Poslanec Petr Fiala""","""2013ps_stenprot_004schuz_s004065.htm""","""/sqw/detail.sqw?id=6074""","""Děkuji. Vážená pane předsedající, vážení členové vlády, dámy a pánové, dovolte mi, abych vám řekl několik slov ke sněmovnímu dokumentu 151, což je poz…",1,"""ps""",2013,4,65,2013-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""4. schůze""",true,true,true,"""6074""",553
"""Poslanec Petr Fiala""","""2013ps_stenprot_004schuz_s004071.htm""","""/sqw/detail.sqw?id=6074""","""Dámy a pánové, odpověděl bych na otázku paní poslankyně Jourové. Chápu paní poslankyni Jourovou. Doufám, že chápat mohu i bez prostřednictví paní před…",1,"""ps""",2013,4,71,2013-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""4. schůze""",true,true,true,"""6074""",264
"""Poslanec Petr Fiala""","""2013ps_stenprot_004schuz_s004071.htm""","""/sqw/detail.sqw?id=6074""","""Děkuji. Na to jsou jednoznačná čísla, kterými to lze doložit, a tyto prostředky jsou prostě potřeba. (Potlesk části poslanců.)""",3,"""ps""",2013,4,71,2013-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""4. schůze""",true,true,true,"""6074""",18
"""Poslanec Petr Fiala""","""2013ps_stenprot_004schuz_s004073.htm""","""/sqw/detail.sqw?id=6074""","""Dámy a pánové. Rád bych prostřednictvím paní předsedající sdělil paní poslankyni Jourové, že pokud zmiňovala nadstandardní spolupráci České republiky …",14,"""ps""",2013,4,73,2013-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""4. schůze""",true,true,true,"""6074""",107
"""Poslanec Petr Fiala""","""2013ps_stenprot_004schuz_s004074.htm""","""/sqw/detail.sqw?id=6074""","""Pokud jde o evropské strukturální fondy a jejich čerpání, mám zkušenosti ze všech rolí, a to z role řešitele, z role příjemce i z role řídícího orgánu…",1,"""ps""",2013,4,74,2013-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""4. schůze""",true,true,false,"""6074""",101
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005002.htm""","""/sqw/detail.sqw?id=6074""","""Děkuji, pane předsedo. Dámy a pánové, je o mně známo, že jsem vcelku důsledným obhájcem parlamentní demokracie, a tudíž bych měl mít radost, že se sch…",2,"""ps""",2013,5,2,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",484
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005004.htm""","""/sqw/detail.sqw?id=6074""","""Vážený pane předsedo, dámy a pánové, pan předseda Filip mě tady pochválil jako politologa, já předpokládám, že to udělal proto, aby hned na začátku os…",5,"""ps""",2013,5,4,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",114
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005005.htm""","""/sqw/detail.sqw?id=6074""","""Pan předseda Faltýnek tady zmiňoval naše chyby a věci, kterých jsme se dopustili, když jsme byli ve vládě. My jsme za ty chyby zaplatili a zaplatili j…",1,"""ps""",2013,5,5,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,false,"""6074""",256
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005008.htm""","""/sqw/detail.sqw?id=6074""","""Vážený pane předsedo, dámy a pánové, celá ta debata mě jenom utvrdila v tom, že můj příměr s řidičákem je naprosto přesný. Pan Babiš je možná dobrý ři…",6,"""ps""",2013,5,8,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",145


In [273]:
df_mluvci.filter(pl.col("jmeno").str.contains("Fiala"))

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""5458""",2006,"""5458_5.html""","""Ing. Jaroslav Fiala, CSc.""","""Karlovarský""","""m""",2006-06-03,2010-06-03,"""ČSSD""",null,"""1956""","[{""člen"",""Výbor pro obranu"",""12. 9. 2006"",""3. 6. 2010""}, {""člen"",""Kontrolní výbor"",""12. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""5. 6. 2006"",""3. 6. 2010""}]",1956,null,"""Jaroslav Fiala""","[""Ing."", ""CSc.""]"
"""5459""",2006,"""5459_5.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2006-06-03,2010-06-03,"""ODS""",null,"""1969""","[{""člen"",""Výbor pro životní prostředí"",""12. 9. 2006"",""3. 6. 2010""}, {""člen"",""Rozpočtový výbor"",""12. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Skupina ČR - Čína"",""21. 3. 2007"",""3. 6. 2010""}]",1969,null,"""Radim Fiala""","[""Ing.""]"
"""5459""",2010,"""5459_6.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2010-05-29,2013-08-28,"""ODS""",null,"""1969""","[{""člen"",""Rozpočtový výbor"",""7. 7. 2010"",""18. 12. 2012""}, {""člen"",""Výbor pro životní prostředí"",""7. 7. 2010"",""28. 8. 2013""}, … {""místopředsed"",""Skupina ČR - Ruská federace"",""3. 2. 2011"",""28. 8. 2013""}]",1969,null,"""Radim Fiala""","[""Ing.""]"
"""5459""",2013,"""5459_7.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2013-10-26,2017-10-26,"""Usvit""",null,"""1969""","[{""člen"",""Rozpočtový výbor"",""4. 12. 2013"",""30. 4. 2015""}, {""člen"",""Organizační výbor"",""4. 12. 2013"",""26. 3. 2014""}, … {""předsed"",""Skupina ČR - Turecká republika"",""13. 2. 2014"",""26. 10. 2017""}]",1969,null,"""Radim Fiala""","[""Ing.""]"
"""6074""",2013,"""6074_7.html""","""Prof. PhDr. Petr Fiala, Ph.D., LL.M.""","""Jihomoravský""","""m""",2013-10-26,2017-10-26,"""ODS""",null,"""1964""","[{""člen"",""Výbor pro evropské záležitosti"",""4. 12. 2013"",""26. 10. 2017""}, {""místopředsed"",""Výbor pro evropské záležitosti"",""5. 12. 2013"",""26. 10. 2017""}, … {""místopředsed"",""Poslanecký klub Občanské demokratické strany"",""6. 11. 2013"",""9. 12. 2014""}]",1964,null,"""Petr Fiala""","[""Prof."", ""PhDr."", … ""LL.M.""]"
"""5459""",2017,"""5459_8.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2017-10-21,2021-10-20,"""SPD""",null,"""1969""","[{""člen"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Hospodářský výbor"",""28. 11. 2017"",""20. 10. 2021""}, … {""člen"",""Skupina ČR - Brazílie"",""18. 3. 2019"",""20. 10. 2021""}]",1969,null,"""Radim Fiala""","[""Ing.""]"
"""6074""",2017,"""6074_8.html""","""Prof. PhDr. Petr Fiala, Ph.D., LL.M.""","""Jihomoravský""","""m""",2017-10-21,2021-10-20,"""ODS""",null,"""1964""","[{""člen"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""místopředsed"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Poslanecký klub Občanské demokratické strany"",""24. 10. 2017"",""20. 10. 2021""}]",1964,null,"""Petr Fiala""","[""Prof."", ""PhDr."", … ""LL.M.""]"
"""5459""",2021,"""5459_9.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2021-10-09,null,"""SPD""",null,"""24. 7. 1969""","[{""člen"",""Hospodářský výbor"",""23. 11. 2021"",null}, {""člen"",""Organizační výbor"",""23. 11. 2021"",null}, … {""předsed"",""Poslanecký klub Svoboda a přímá demokracie"",""12. 10. 2021"",null}]",1969,1969-07-24,"""Radim Fiala""","[""Ing.""]"
"""6074""",2021,"""6074_9.html""","""Prof. PhDr. Petr Fiala, Ph.D., LL.M.""","""Jihomoravský""","""m""",2021-10-09,null,"""SPOLU""","""ODS""","""1. 9. 1964""","[{""člen"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",null}, {""předsed"",""Vláda České republiky"",""28. 11. 2021"",null}]",1964,1964-09-01,"""Petr Fiala""","[""Prof."", ""PhDr."", … ""LL.M.""]"


In [274]:
df.filter(pl.col("mluvci").str.contains("Petr Fiala"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_038schuz_s038048.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní předsedkyně, dámy a pánové, Mezinárodní úmluva proti dopingu ve sportu patří do kategorie smluv, jejichž přijetí nebo přijetí jejich změn …",6,"""ps""",2010,38,48,2012-05-03,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""38. schůze""",true,true,true,null,222
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040146.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní předsedající, dámy a pánové, dovolte mi seznámit vás ve stručnosti s návrhem novely zákona o výkonu ústavní výchovy a ochranné výchovy ve …",10,"""ps""",2010,40,146,2012-06-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,475
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040147.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Ano, paní předsedající, já souhlasím s tímto návrhem na zkrácení lhůty na 48 hodin.""",8,"""ps""",2010,40,147,2012-06-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,13
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040244.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní poslankyně, dámy a pánové, mám tu čest stát před vámi při interpelacích poprvé, tak mi dovolte na úvod jenom krátkou poznámku. Když jsem p…",5,"""ps""",2010,40,244,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,531
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040244.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Ano, děkuji, pane předsedající, nyní odpovím velmi stručně. Jsem si plně vědom problematické situace v oblasti financování regionálního školství. Jsem…",9,"""ps""",2010,40,244,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,110
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040245.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážený pane poslanče, dámy a pánové, děkuji za váš dotaz, který nás přivádí k jiné části širokého resortu, který mám tu čest spravovat, totiž k oblast…",6,"""ps""",2010,40,245,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,323
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040246.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Paní Hana Zejdová se sice v roce 1993 umístila na prvním místě na mistrovství Evropy v bezmotorovém létání, v následujících letech už nebyla tak úspěš…",1,"""ps""",2010,40,246,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,false,null,145
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040246.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""To vysvětlení, pane poslanče, které jsem poskytl, v žádném případě nezpochybňuje jedinečnost výkonů paní Zejdové ani to její momentální úsilí. Podstat…",5,"""ps""",2010,40,246,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,

## Sloučení

In [276]:
df = df.join(df_mluvci, on=['id','obdobi'], how='left', nulls_equal=False)

In [277]:
df.filter(pl.col("mluvci").str.contains("Zaorálek"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Pan poslanec Zaorálek Lubomír""","""1996ps_stenprot_001schuz_s001001.htm""","""/sqw/hlasy.sqw?G=5978""","""Slibuji.""",189,"""ps""",1996,1,1,1996-06-25,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""1. schůze""",true,true,true,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_005schuz_s005049.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, vážená sněmovno, vážení poslanci, dovolte, abych vaši pozornost obrátil zpět k rozpočtu. Já bych tady chtěl vyjádřit, co cít…",6,"""ps""",1996,5,49,1996-10-09,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""5. schůze""",true,true,true,null,562,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_005schuz_s005056.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, vážení poslanci, chtěl bych k tomuto návrhu smlouvy mezi Českou a Polskou republikou o vzájemné pomoci v celních otázkách po…",12,"""ps""",1996,5,56,1996-10-10,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""5. schůze""",true,true,true,null,132,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_005schuz_s005057.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, vážení poslanci, já se domnívám, že je těžkým dluhem minulosti, který s sebou neseme, izolacionismus české zahraniční politi…",17,"""ps""",1996,5,57,1996-10-10,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""5. schůze""",true,true,true,null,259,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_007schuz_s007011.htm""","""/sqw/hlasy.sqw?G=5978""","""Pane předsedající, vážená vládo, vážení poslanci, dovolte mi, abych se ještě jednou vrátil k tématu, které zde dnes už několikrát kolovalo, a to je ke…",2,"""ps""",1996,7,11,1996-12-11,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""7. schůze""",true,true,true,null,565,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_007schuz_s007013.htm""","""/sqw/hlasy.sqw?G=5978""","""vážený pane předsedající vážená vládo, vážené poslankyně a poslanci, dovolte, abych ještě předložil pozměňovací návrh k pozměňovacímu návrhu pana posl…",2,"""ps""",1996,7,13,1996-12-11,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""7. schůze""",true,true,true,null,42,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_007schuz_s007019.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, vážená sněmovno, zahraniční výbor skutečně projednal na své 4. schůzi, která se konala 16. října, tento návrh, jak mu bylo u…",30,"""ps""",1996,7,19,1996-12-12,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""7. schůze""",true,true,true,null,68,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_007schuz_s007021.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, vážená sněmovno, dovolím si jen upozornit na to, že tu existuje určitý rozdíl mezi tím usnesením, které navrhuje výbor pro z…",31,"""ps""",1996,7,21,1996-12-12,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""7. schůze""",true,true,true,null,512,null,null,null,null,null,null,null,

In [278]:
df_mluvci.filter(pl.col('id') == '312')

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""312""",1996,"""312_2.html""","""JUDr. Zuzka Bebarová Rujbrová""","""Jihomoravský""","""f""",1996-06-01,1998-06-19,"""KSČM""",null,"""1951""","[{""člen"",""Ústavně právní výbor"",""2. 7. 1996"",""19. 6. 1998""}, {""člen"",""Podvýbor pro zpravodajské služby"",""13. 3. 1997"",""19. 6. 1998""}, … {""místopředsed"",""Poslanecký klub Komunistické strany Čech a Moravy"",""4. 6. 1996"",""19. 6. 1998""}]",1951,null,"""Zuzka Bebarová Rujbrová""","[""JUDr.""]"
"""312""",1998,"""312_3.html""","""JUDr. Zuzka Bebarová Rujbrová""","""Jihomoravský""","""f""",1998-06-20,2002-06-20,"""KSČM""",null,"""1951""","[{""člen"",""Ústavně právní výbor"",""22. 7. 1998"",""28. 1. 2000""}, {""člen"",""Výbor pro veřejnou správu, regionální rozvoj a životní prostředí"",""28. 1. 2000"",""20. 6. 2002""}, … {""člen"",""Skupina ČR - Kazachstán"",""2. 7. 1999"",""20. 6. 2002""}]",1951,null,"""Zuzka Bebarová Rujbrová""","[""JUDr.""]"
"""312""",2002,"""312_4.html""","""JUDr. Zuzka Bebarová Rujbrová""","""Jihomoravský""","""f""",2002-06-15,2006-06-15,"""KSČM""",null,"""1951""","[{""člen"",""Mandátový a imunitní výbor"",""11. 7. 2002"",""15. 6. 2006""}, {""člen"",""Petiční výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Dozorčí rada České konsolidační agentury"",""15. 3. 2006"",""9. 5. 2007""}]",1951,null,"""Zuzka Bebarová Rujbrová""","[""JUDr.""]"
"""312""",2006,"""312_5.html""","""JUDr. Zuzka Bebarová Rujbrová""","""Jihomoravský""","""f""",2006-06-03,2010-06-03,"""KSČM""",null,"""1951""","[{""člen"",""Výbor pro bezpečnost"",""12. 9. 2006"",""3. 6. 2010""}, {""člen"",""Petiční výbor"",""12. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Dozorčí rada České konsolidační agentury"",""15. 3. 2006"",""9. 5. 2007""}]",1951,null,"""Zuzka Bebarová Rujbrová""","[""JUDr.""]"
"""312""",2010,"""312_6.html""","""JUDr. Zuzka Bebarová Rujbrová""","""Jihomoravský""","""f""",2010-05-29,2013-08-28,"""KSČM""",null,"""1951""","[{""člen"",""Výbor pro obranu a bezpečnost"",""7. 7. 2010"",""6. 12. 2011""}, {""člen"",""Výbor pro bezpečnost"",""13. 12. 2011"",""28. 8. 2013""}, … {""člen"",""Skupina ČR - Vietnam"",""16. 12. 2010"",""28. 8. 2013""}]",1951,null,"""Zuzka Bebarová Rujbrová""","[""JUDr.""]"
"""312""",2013,"""312_7.html""","""JUDr. Zuzka Bebarová Rujbrová""","""Jihomoravský""","""f""",2013-10-26,2017-10-26,"""KSČM""",null,"""1951""","[{""člen"",""Petiční výbor"",""4. 12. 2013"",""26. 10. 2017""}, {""člen"",""Výbor pro bezpečnost"",""4. 12. 2013"",""26. 10. 2017""}, … {""člen"",""Skupina ČR - Kuba"",""12. 1. 2016"",""26. 10. 2017""}]",1951,null,"""Zuzka Bebarová Rujbrová""","[""JUDr.""]"


In [279]:
df.filter(pl.col("mluvci").str.contains("Zaorálek")).filter(pl.col('gender') == 'f').select(pl.col(['mluvci','mluvci_id','jmeno','id','gender','soubor'])).unique()

mluvci,mluvci_id,jmeno,id,gender,soubor
str,str,str,str,str,str
"""Předseda PS Lubomír Zaorálek""","""/sqw/detail.sqw?id=312""","""JUDr. Zuzka Bebarová Rujbrová""","""312""","""f""","""2002ps_stenprot_001schuz_s001012.htm"""


In [280]:
klic = df_mluvci.select(pl.col(["jmeno","id"])).unique()
klic.sample(3)

jmeno,id
str,str
"""Ing. et Ing. Leo Luzar""","""6279"""
"""Tomio Okamura""","""6105"""
"""Tomáš Hanzel""","""6452"""


In [281]:
df.group_by('obdobi').agg(pl.col('pocet_slov').sum()).sort(by='obdobi')

obdobi,pocet_slov
i32,u32
1993,3681289
1996,4029495
1998,8963383
2002,9397989
2006,7232451
2010,6565130
2013,11227634
2017,13069642
2021,15589023


In [282]:
df.filter(pl.col('obdobi') == 2021).group_by('id').agg(pl.col('pocet_slov').sum()).sort(by='pocet_slov',descending=True).join(klic, how='left',on='id')

id,pocet_slov,jmeno
str,u32,str
null,2016526,null
"""6105""",1132243,"""Tomio Okamura"""
"""6150""",1095221,"""Ing. Andrej Babiš"""
"""6621""",527978,"""doc. Ing. Karel Havlíček, Ph.D., MBA"""
"""6545""",511438,"""JUDr. Alena Schillerová, Ph.D."""
"""6487""",391630,"""Ing. Patrik Nacher"""
"""6473""",384308,"""PhDr. Olga Richterová, Ph.D."""
"""6463""",369745,"""Ing. Aleš Juchelka"""
"""6477""",343292,"""Mgr. et Mgr. Jakub Michálek, Ph.D."""


In [283]:
df.filter(pl.col("id").is_null()).group_by("mluvci").agg(pl.col("datum").unique().len()).sort(by="datum",descending=True)

mluvci,datum
str,u32
"""Poslanec Vojtěch Filip""",314
"""Poslanec Václav Exner""",304
"""Místopředsedkyně PSP Petra Buzková""",280
"""Poslanec Dalibor Matulka""",268
"""Poslanec Miloslav Výborný""",236
"""Poslanec Karel Vymětal""",227
"""Poslanec Jiří Payne""",226
"""Poslanec Vlastimil Tlustý""",219
"""Místopředseda PSP Ivan Langer""",216


In [284]:
df.filter(pl.col("mluvci").str.contains("Vojtěch Filip")).group_by("mluvci_id").agg(pl.col("text").unique().len()).sort(by="text",descending=True)

mluvci_id,text
str,u32
"""/sqw/detail.sqw?id=303""",29259
"""/sqw/p.sqw?P=452""",925
"""/sqw/hlasy.sqw?G=10693""",219
"""/sqw/hlasy.sqw?G=5978""",158
"""/sqw/hlasy.sqw?G=8471""",84
"""/sqw/detail.sqw?id=48&o=2""",60
"""/ff/ab/33/01.htm""",41
"""/cgi-bin/win/sqw/hlasy.sqw?G=11995""",36
"""/sqw/historie.sqw?T=1&O=3""",26


In [285]:
df.sort(by='datum').tail(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Poslanec Tomáš Müller""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6772""","""Za šesté, pozměňovací návrh C poslankyně Peštové, sněmovní dokument 6828. Stanovisko výboru: doporučující.""",9,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6772""",12,"""6772_9.html""","""Ing. Tomáš Müller""","""Olomoucký""","""m""",2021-10-09,null,"""PirSTAN""","""STAN""","""28. 4. 1966""","[{""člen"",""Rozpočtový výbor"",""23. 11. 2021"",""23. 3. 2022""}, {""člen"",""Petiční výbor"",""23. 11. 2021"",null}, … {""člen"",""Skupina ČR - Rakousko"",""2. 3. 2022"",null}]",1966,1966-04-28,"""Tomáš Müller""","[""Ing.""]"
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6178""","""Pan ministr? (Ministr: Neutrální.) Zahajuji hlasování. Ptám se, kdo je pro? Ať zmáčkne tlačítko a zvedne ruku. Kdo je proti? Hlasování s pořadovým čís…",10,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6178""",40,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Thajské království"",""4. 4. 2025"",null}]",1971,1971-11-10,"""Jan Bartošek""","[""Ing.""]"
"""Poslanec Tomáš Müller""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6772""","""Protože tento pozměňovací návrh C nebyl schválen, nyní budeme hlasovat o pozměňovacím návrhu D poslance Smetany, sněmovní dokument 6825. Stanovisko vý…",11,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6772""",21,"""6772_9.html""","""Ing. Tomáš Müller""","""Olomoucký""","""m""",2021-10-09,null,"""PirSTAN""","""STAN""","""28. 4. 1966""","[{""člen"",""Rozpočtový výbor"",""23. 11. 2021"",""23. 3. 2022""}, {""člen"",""Petiční výbor"",""23. 11. 2021"",null}, … {""člen"",""Skupina ČR - Rakousko"",""2. 3. 2022"",null}]",1966,1966-04-28,"""Tomáš Müller""","[""Ing.""]"
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6178""","""Pan ministr? (Ministr: Neutrální.) Zahajuji hlasování. Ptám se, kdo je pro? Ať zmáčkne tlačítko a zvedne ruku. Kdo je proti? Hlasování s pořadovým čís…",12,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6178""",40,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Thajské království"",""4. 4. 2025"",null}]",1971,1971-11-10,"""Jan Bartošek""","[""Ing.""]"
"""Poslanec Tomáš Müller""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6772""","""Za osmé, pozměňovací návrh E poslankyně Maříkové, sněmovní dokument 6801. Stanovisko výboru: bez stanoviska.""",13,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6772""",13,"""6772_9.html""","""Ing. Tomáš Müller""","""Olomoucký""","""m""",2021-10-09,null,"""PirSTAN""","""STAN""","""28. 4. 1966""","[{""člen"",""Rozpočtový výbor"",""23. 11. 2021"",""23. 3. 2022""}, {""člen"",""Petiční výbor"",""23. 11. 2021"",null}, … {""člen"",""Skupina ČR - Rakousko"",""2. 3. 2022"",null}]",1966,1966-04-28,"""Tomáš Müller""","[""Ing.""]"
"""M

In [286]:
df.filter(pl.col("id") == "6105").sort(by="datum").group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum())

datum,pocet_slov
date,u32
2013-12-10,258
2013-12-12,595
2014-02-18,618
2014-03-19,1938
2014-03-25,4767
2014-04-30,1466
2014-05-15,2377
2014-06-10,1477
2014-06-11,2252


In [287]:
alt.Chart(
    df.filter(pl.col("mluvci").str.contains("Andrej Babiš")).sort(by="datum").group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [288]:
alt.Chart(
    df.filter(pl.col("mluvci").str.contains("Tomio Okamura")).sort(by="datum").group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [289]:
alt.Chart(
    df.sort(by="datum").group_by_dynamic(index_column="datum",every="1mo").agg(pl.col("pocet_slov").sum()).to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [290]:
alt.Chart(
    df.filter(pl.col("mluvci").str.contains("Marek Benda")).sort(by="datum").group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [291]:
df.filter(pl.col('obdobi') == 2021).filter(pl.col('id').is_null()).select(pl.col('mluvci')).unique()

mluvci
str
"""Ministr zahraničních věcí ČR Jan Lipavský"""
"""Předsedkyně PSP Markéta Pekarová Adamová"""
"""Minulý"""
"""Místopředseda vlády a ministr pro místní rozvoj ČR Ivan Bartoš"""
"""Ministr zemědělství ČR Marek Výborný"""
"""Prezident České republiky Petr Pavel"""
"""Poslanec Matěj Ondřej Havel"""
"""Ministr dopravy ČR Martin Kupka"""
"""Ministr průmyslu a obchodu ČR Jozef Síkela"""


In [292]:
df.filter(pl.col('obdobi') == 2021).filter(pl.col('id').is_null())

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Ministr obrany ČR Lubomír Metnar""","""2021ps_stenprot_001schuz_s001009.htm""","""https://www.vlada.cz/cz/clenove-vlady/lubomir-metnar-167028/""","""Pane předsedo, děkuji. Já jenom technická. Hlasoval jsem pro, ale nepodařilo se mi přihlásit. Tak to jenom na upřesnění.""",3,"""ps""",2021,1,9,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,18,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředseda vlády, ministr průmyslu a obchodu a ministr dopravy ČR Karel Havlíček""","""2021ps_stenprot_001schuz_s001010.htm""","""https://www.vlada.cz/cz/clenove-vlady/karel-havlicek-173244/""","""Vážený pane předsedající, vážené paní poslankyně, vážení páni poslanci. Nebudu v tuto chvíli mluvit k těm právním náležitostem, určitě o tom ještě v p…",5,"""ps""",2021,1,10,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,266,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředseda vlády, ministr průmyslu a obchodu a ministr dopravy ČR Karel Havlíček""","""2021ps_stenprot_001schuz_s001011.htm""","""https://www.vlada.cz/cz/clenove-vlady/karel-havlicek-173244/""","""Můžete se nám vysmát a věcně argumentovat tím, že jste nám nabídli dva místopředsedy, případně pár výborů, a to zase my můžeme reagovat, že s místopře…",1,"""ps""",2021,1,11,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,false,null,634,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředsedkyně vlády a ministryně financí ČR Alena Schillerová""","""2021ps_stenprot_001schuz_s001011.htm""","""https://www.vlada.cz/cz/clenove-vlady/alena-schillerova-167019/""","""Děkuji za slovo. Vážený pane předsedající, vážené paní poslankyně, vážení páni poslanci. Především bych vám všem chtěla pogratulovat ke zvolení a záro…",3,"""ps""",2021,1,11,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,541,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředsedkyně vlády a ministryně financí ČR Alena Schillerová""","""2021ps_stenprot_001schuz_s001012.htm""","""https://www.vlada.cz/cz/clenove-vlady/alena-schillerova-167019/""","""Výkladu čl. 32 v tom smyslu, že se jedná o ustanovení o neslučitelnosti, nikoliv o nevolitelnosti, svědčí také ustanovení čl. 2 odst. 4 a čl. 2 odst. …",1,"""ps""",2021,1,12,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,false,null,526,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Ministr obrany ČR Lubomír Metnar""","""2021ps_stenprot_001schuz_s001012.htm""","""https://www.vlada.cz/cz/clenove-vlady/lubomir-metnar-167028/""","""Děkuji za slovo. Vážený pane předsedající, vážené paní poslankyně, vážení páni poslanci, já se nebudu vyjadřovat k právním otázkám, já se vyjádřím k d…",3,"""ps""",2021,1,12,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,459,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Ministr obrany ČR Lubomír Metnar""","""2021ps_stenprot_001schuz_s001013.htm""","""https://www.vlada.cz/cz/clenove-vlady/lubomir-metnar-167028/""","""I pokud se jedná o otázku ŘSD, jsou zde také vidět silné a věcné kroky docenta Havlíčka, zejména při schvalování záměru na transformaci na novou státn…",1,"""ps""",2021,1,13,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,false,null,185,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Ministryně pro místní rozvoj ČR Klára Dostálová""","""2021ps_stenprot_001schuz_s001013.htm""","""https://www.vlada.cz/cz

In [293]:
df.filter(pl.col('obdobi') == 2021).filter(pl.col('id') == '6105')

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Místopředseda PSP Tomio Okamura""","""2021ps_stenprot_001schuz_s001001.htm""","""/sqw/detail.sqw?id=6105""","""(Schůze zahájena ve 14.00 hodin.)""",1,"""ps""",2021,1,1,2021-11-08,"""PČR, PS 2021-...""","""1. schůze""",true,true,false,"""6105""",4,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002001.htm""","""/sqw/detail.sqw?id=6105""","""Vážené dámy a pánové, navrhuji jako první bod dnešní schůze Sněmovny mimořádný bod programu s názvem Diskuse o nepřijatelných opatřeních souvisejících…",5,"""ps""",2021,2,1,2021-11-23,"""PČR, PS 2021-...""","""2. schůze""",true,true,true,"""6105""",129,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002002.htm""","""/sqw/detail.sqw?id=6105""","""Já osobně k tomu dodávám, že omezování svobody, omezování pohybu, šikanování podnikatelů a občanů navíc poškozuje občany i na jejich zdraví. Pokud se …",1,"""ps""",2021,2,2,2021-11-23,"""PČR, PS 2021-...""","""2. schůze""",true,true,false,"""6105""",1295,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002003.htm""","""/sqw/detail.sqw?id=6105""","""Dovolte dále citovat nález rozšířeného senátu Nejvyššího správního soudu. Cituji: ""Abyste mohli zasahovat do základních práv v důsledku mimořádných op…",1,"""ps""",2021,2,3,2021-11-23,"""PČR, PS 2021-...""","""2. schůze""",true,true,false,"""6105""",1137,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002004.htm""","""/sqw/detail.sqw?id=6105""","""Navíc vláda Andreje Babiše s účinností od včerejšího dne zavádí takzvaný bavorský model, podle kterého zdraví neočkovaní lidé nesmí chodit na volnočas…",1,"""ps""",2021,2,4,2021-11-23,"""PČR, PS 2021-...""","""2. schůze""",true,true,false,"""6105""",372,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002004.htm""","""/sqw/detail.sqw?id=6105""","""Ne, ne, já jsem říkal, že nejprve bych to navrhl - já jsem dal dvě varianty... (Pekarová: Ano, prosím.) ...abych dal prostor k tomu po

In [294]:
df.sample(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Místopředseda PSP Petr Fiala""","""2017ps_stenprot_087schuz_s087024.htm""","""/sqw/detail.sqw?id=6074""","""Děkuji. Pan poslanec Dominik Feri, faktická poznámka, poté pan poslanec Martin Jiránek. Prosím, máte slovo.""",4,"""ps""",2017,87,24,2021-03-02,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""87. schůze""",true,true,true,"""6074""",14,"""6074_8.html""","""Prof. PhDr. Petr Fiala, Ph.D., LL.M.""","""Jihomoravský""","""m""",2017-10-21,2021-10-20,"""ODS""",null,"""1964""","[{""člen"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""místopředsed"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Poslanecký klub Občanské demokratické strany"",""24. 10. 2017"",""20. 10. 2021""}]",1964,null,"""Petr Fiala""","[""Prof."", ""PhDr."", … ""LL.M.""]"
"""Poslankyně Kateřina Dostálová""","""2002ps_stenprot_042schuz_s042034.htm""","""/sqw/detail.sqw?id=358""","""Děkuji. Vážená paní předsedající, kolegyně a kolegové, pan ministr jako předkladatel tohoto návrhu ukončil svou závěrečnou řeč slovy, že se nebrání di…",6,"""ps""",2002,42,34,2005-03-23,"""PČR, PS 2002-2006""",null,true,true,true,"""358""",117,"""358_4.html""","""Kateřina Dostálová""","""Olomoucký""","""f""",2002-06-15,2006-06-15,"""ODS""",null,"""1964""","[{""člen"",""Petiční výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""člen"",""Výbor pro vědu, vzdělání, kulturu, mládež a tělovýchovu"",""16. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Skupina ČR - Ukrajina"",""17. 4. 2003"",""15. 6. 2006""}]",1964,null,"""Kateřina Dostálová""",[]
"""Poslanec Jaromír Schling""","""2002ps_stenprot_003schuz_s003091.htm""","""/sqw/detail.sqw?id=234""","""(10.10 hodin) Naproti tomu jako špatný příklad mohu uvést komisi, která přešetřovala činnost transplantačního centra. Jejím výsledkem na jedné straně…",1,"""ps""",2002,3,91,2002-08-08,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""3. schůze""",true,true,false,"""234""",215,"""234_4.html""","""Ing. Jaromír Schling""","""Vysočina""","""m""",2002-06-15,2006-06-15,"""ČSSD""",null,"""1946""","[{""člen"",""Hospodářský výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""člen"",""Výbor pro evropské záležitosti"",""24. 9. 2004"",""15. 6. 2006""}, … {null,""Vláda České republiky"",""27. 4. 2000"",""14. 7. 2002""}]",1946,null,"""Jaromír Schling""","[""Ing.""]"
"""Poslankyně Marie Rusová""","""2002ps_stenprot_037schuz_s037049.htm""","""/sqw/detail.sqw?id=5296""","""Vážená paní předsedající, dámy a pánové, v souvislosti s vydáním nového trestního zákona se zavádí trestní odpovědnost právnických osob. Jde o převrat…",3,"""ps""",2002,37,49,2004-11-02,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""37. schůze""",true,true,true,"""5296""",656,"""5296_4.html""","""JUDr. Marie Rusová""","""Vysočina""","""f""",2002-06-15,2006-06-15,"""KSČM""",null,"""1950""","[{""člen"",""Ústavně právní výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""místopředsed"",""Ústavně právní výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""místopředsed"",""Skupina ČR - Francie"",""14. 5. 2004"",""15. 6. 2006""}]",1950,null,"""Marie Rusová""","[""JUDr.""]"
"""Místopředseda PSP František Brožík""","""1998ps_stenprot_025schuz_s025084.htm""","""/sqw/p.sqw?P=432""","""Děkuji panu poslanci Kučerovi. Jako další se přihlásil pan poslanec Stanislav Šustr, kterému udílím slovo.""",8,"""ps""",1998,25,84,2000-05-17,"""Parlament České republiky, Poslanecká sněmovna 1998 – 2002""","""25. schůze""",true,true,true,null,14,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""M

In [295]:
df.select(pl.col("pocet_slov")).sum()

pocet_slov
u32
79756036


"Naše pilotní studie ukazuje, že normální řeč plynula rychlostí 1,75 slov za sekundu (105 slov/min.). Bylo ji však možné zrychlit při rutinních úlohách (mě- síce) až na 3 slova za sekundu (180 slov/min.).
"

In [297]:
df.select(pl.col("pocet_slov")).sum().item() / (105 * 60 * 8 * 365.25)

4.33254218137175

## Kdo nejvíc mluvil

In [299]:
df.filter(pl.col("obdobi") == 2021).group_by("mluvci").agg(pl.col("pocet_slov").median()).sort(by="pocet_slov",descending=True)

mluvci,pocet_slov
str,f64
"""Předseda Národní rozpočtové rady Mojmír Hampl""",1055.0
"""Poslanec Andrej Babiš""",982.5
"""Senátor Stanislav Balík""",815.0
"""Prezident České republiky Miloš Zeman""",756.5
"""Předseda vlády ČR Andrej Babiš""",715.0
"""Poslankyně Markéta Pekarová Adamová""",688.5
"""Ministr zdravotnictví ČR Adam Vojtěch""",643.5
"""Poslanec Tomio Okamura""",643.5
"""Senátor Jan Grulich""",611.0


In [300]:
df.filter(pl.col("obdobi") == 2021).group_by("mluvci").agg(pl.col("pocet_slov").sum()).sort(by="pocet_slov",descending=True)

mluvci,pocet_slov
str,u32
"""Poslanec Tomio Okamura""",1132239
"""Poslanec Andrej Babiš""",1095221
"""Místopředseda PSP Karel Havlíček""",518570
"""Poslankyně Alena Schillerová""",511438
"""Poslanec Patrik Nacher""",391630
"""Místopředsedkyně PSP Olga Richterová""",384269
"""Poslanec Jakub Michálek""",343292
"""Poslanec Radim Fiala""",314323
"""Místopředseda PSP Jan Bartošek""",313308


In [301]:
soucasne_poslanectvo = df_mluvci.filter(pl.col("obdobi") == 2021).select(pl.col("jmeno_ciste")).to_series().to_list()
soucasne_poslanectvo[0:5]

['Jaroslav Bašta',
 'Petr Bendl',
 'Marek Benda',
 'Pavel Žáček',
 'Antonín Tesařík']

In [302]:
statistiky_soucasneho_poslanectva = []
for p in soucasne_poslanectvo:
    statistiky_soucasneho_poslanectva.append(
        {
            'jmeno':p,
            'pocet_slov':df.filter((pl.col('obdobi') == 2021) & (pl.col('mluvci').str.contains(p))).select(pl.col('pocet_slov')).sum().item()
        }
    )

In [303]:
pl.DataFrame(statistiky_soucasneho_poslanectva).sort(by='pocet_slov',descending=True)

jmeno,pocet_slov
str,i64
"""Tomio Okamura""",1132243
"""Andrej Babiš""",1099566
"""Karel Havlíček""",535753
"""Alena Schillerová""",524800
"""Patrik Nacher""",391630
"""Olga Richterová""",384308
"""Aleš Juchelka""",369745
"""Jakub Michálek""",343292
"""Marian Jurečka""",321921


## Vlády

In [305]:
import pandas as pd

In [306]:
vlady = [
    {'vlada': 'první vláda Václava Klause', 'vlada_od': '2. 7. 1992', 'vlada_do': '4. 7. 1996'},
    {'vlada': 'druhá vláda Václava Klause', 'vlada_od': '4. 7. 1996', 'vlada_do': '2. 1. 1998'},
    {'vlada': 'vláda Josefa Tošovského', 'vlada_od': '2. 1. 1998', 'vlada_do': '22. 7. 1998'},
    {'vlada': 'vláda Miloše Zemana', 'vlada_od': '22. 7. 1998', 'vlada_do': '15. 7. 2002'},
    {'vlada': 'vláda Vladimíra Špidly', 'vlada_od': '15. 7. 2002', 'vlada_do': '4. 8. 2004'},
    {'vlada': 'vláda Stanislava Grosse', 'vlada_od': '4. 8. 2004', 'vlada_do': '25. 4. 2005'},
    {'vlada': 'vláda Jiřího Paroubka', 'vlada_od': '25. 4. 2005', 'vlada_do': '4. 9. 2006'},
    {'vlada': 'první vláda Mirka Topolánka', 'vlada_od': '4. 9. 2006', 'vlada_do': '9. 1. 2007'},
    {'vlada': 'druhá vláda Mirka Topolánka', 'vlada_od': '9. 1. 2007', 'vlada_do': '8. 5. 2009'},
    {'vlada': 'vláda Jana Fischera', 'vlada_od': '8. 5. 2009', 'vlada_do': '13. 7. 2010'},
    {'vlada': 'vláda Petra Nečase', 'vlada_od': '13. 7. 2010', 'vlada_do': '10. 7. 2013'},
    {'vlada': 'vláda Jiřího Rusnoka', 'vlada_od': '10. 7. 2013', 'vlada_do': '29. 1. 2014'},
    {'vlada': 'vláda Bohuslava Sobotky', 'vlada_od': '29. 1. 2014', 'vlada_do': '13. 12. 2017', 'koalice_kandidatka':["ANO2011","KDU-ČSL","ČSSD"],'koalice_navrhovatel':[],'opozice_kandidatka':["KSČM","ODS","TOP09","Usvit"], 'opozice_navrhovatel':[]},
    {'vlada': 'první vláda Andreje Babiše', 'vlada_od': '13. 12. 2017', 'vlada_do': '27. 6. 2018', 'koalice_kandidatka':["ANO2011"],'koalice_navrhovatel':[],'opozice_kandidatka':["ČSSD","TOP09","KDU-ČSL","STAN","Piráti","ODS","SPD","KSČM"],  'opozice_navrhovatel':[]},
    {'vlada': 'druhá vláda Andreje Babiše', 'vlada_od': '27. 6. 2018', 'vlada_do': '17. 12. 2021', 'koalice_kandidatka':["ČSSD","ANO2011"],'koalice_navrhovatel':[],'opozice_kandidatka':["TOP09","KDU-ČSL","STAN","Piráti","ODS","SPD","KSČM"], 'opozice_navrhovatel':[]},
    {'vlada': 'vláda Petra Fialy s Piráty', 'vlada_od': '17. 12. 2021', 'vlada_do': '30. 9. 2024', 'koalice_kandidatka':['SPOLU'],'koalice_navrhovatel':['STAN','ODS','KDU-ČSL','TOP09','Piráti'], 'opozice_kandidatka':['ANO2011','SPD'],'opozice_navrhovatel':[]},
    {'vlada': 'vláda Petra Fialy bez Pirátů', 'vlada_od': '1. 10. 2024', 'vlada_do': '30. 9. 2025', 'koalice_kandidatka':['SPOLU'],'koalice_navrhovatel':['STAN','ODS','KDU-ČSL','TOP09'],'opozice_kandidatka':['ANO2011','SPD'],'opozice_navrhovatel':['Piráti']},
]

vlady = pl.DataFrame(
    vlady, infer_schema_length=None
).with_columns(
    pl.col(['vlada_od','vlada_do']).str.to_date()
).with_columns(
    pl.col(['koalice_kandidatka','koalice_navrhovatel','opozice_kandidatka','opozice_navrhovatel']).list.join(separator="*")
)

vlady = vlady.group_by(
    "vlada"
).agg(
    pl.date_range(pl.col("vlada_od"), pl.col("vlada_do"), "1d").alias("datum")
).explode(
    "datum"
).join(
    vlady, how='left', on='vlada'
).filter(
    ~pl.col('koalice_kandidatka').is_null()
)

In [307]:
vlady.sample(10)

vlada,datum,vlada_od,vlada_do,koalice_kandidatka,koalice_navrhovatel,opozice_kandidatka,opozice_navrhovatel
str,date,date,date,str,str,str,str
"""vláda Petra Fialy s Piráty""",2022-11-23,2021-12-17,2024-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09*Piráti""","""ANO2011*SPD""",""""""
"""druhá vláda Andreje Babiše""",2021-06-05,2018-06-27,2021-12-17,"""ČSSD*ANO2011""","""""","""TOP09*KDU-ČSL*STAN*Piráti*ODS*SPD*KSČM""",""""""
"""druhá vláda Andreje Babiše""",2019-07-15,2018-06-27,2021-12-17,"""ČSSD*ANO2011""","""""","""TOP09*KDU-ČSL*STAN*Piráti*ODS*SPD*KSČM""",""""""
"""druhá vláda Andreje Babiše""",2020-04-26,2018-06-27,2021-12-17,"""ČSSD*ANO2011""","""""","""TOP09*KDU-ČSL*STAN*Piráti*ODS*SPD*KSČM""",""""""
"""vláda Bohuslava Sobotky""",2017-09-24,2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""",""""""
"""vláda Bohuslava Sobotky""",2016-06-18,2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""",""""""
"""vláda Bohuslava Sobotky""",2015-08-08,2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""",""""""
"""vláda Bohuslava Sobotky""",2016-05-26,2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""",""""""
"""vláda Bohuslava Sobotky""",2017-05-03,2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""",""""""


In [308]:
df_vlady = df.filter(
    pl.col("obdobi") >= 2013
).join(vlady, how='left', on='datum').with_columns(
    pl.when(
        pl.col('mluvci_id').str.contains('vlada.cz')
    ).then(
        pl.lit('vláda')
    ).when(
        pl.col('koalice_kandidatka').str.contains(pl.col('kandidatka')) | pl.col('koalice_kandidatka').str.contains(pl.col('navrhovatel'))
    ).then(
        pl.lit('vláda')
    ).when(
        pl.col('koalice_navrhovatel').str.contains(pl.col('navrhovatel')) | pl.col('koalice_navrhovatel').str.contains(pl.col('kandidatka'))
    ).then(
        pl.lit('vláda')
    ).when(
        pl.col('opozice_kandidatka').str.contains(pl.col('kandidatka')) | pl.col('opozice_kandidatka').str.contains(pl.col('navrhovatel'))
    ).then(
        pl.lit('opozice')
    ).when(
        pl.col('opozice_navrhovatel').str.contains(pl.col('navrhovatel')) | pl.col('opozice_navrhovatel').str.contains(pl.col('kandidatka'))
    ).then(
        pl.lit('opozice')
    ).alias('vlada_opo')
).sort(
    by='datum'
)

In [309]:
df_vlady.group_by(['obdobi','vlada_opo']).agg(pl.col('pocet_slov').sum()).sort(by=['obdobi','vlada_opo']).pivot(index="obdobi",on='vlada_opo')

obdobi,null,opozice,vláda
i32,u32,u32,u32
2013,655519,6037683,4534432
2017,620098,8686865,3801173
2021,405710,10358151,4825162


In [310]:
df_vlady.group_by(['obdobi','vlada_opo']).agg(pl.col('pocet_slov').sum()).sort(by=['obdobi']).pivot(
    index='obdobi',on='vlada_opo'
).with_columns(
    (pl.col("null") / (pl.col("null") + pl.col("vláda") + pl.col("opozice"))).alias("podil_null")
).with_columns(
    (pl.col("opozice") / (pl.col("null") + pl.col("vláda") + pl.col("opozice"))).alias("podíl_opozice")
)

obdobi,opozice,vláda,null,podil_null,podíl_opozice
i32,u32,u32,u32,f64,f64
2013,6037683,4534432,655519,0.058384,0.537752
2017,8686865,3801173,620098,0.047306,0.662708
2021,10358151,4825162,405710,0.026025,0.664452


In [311]:
df_vlady.group_by(['obdobi','vlada_opo']).agg(pl.col('id').unique().len()).sort(by=['obdobi']).pivot(
    index='obdobi',on='vlada_opo'
)

obdobi,vláda,null,opozice
i32,u32,u32,u32
2013,110,163,92
2017,102,75,130
2021,127,20,125


In [312]:
df_vlady.filter((pl.col('obdobi') == 2021) & (pl.col('vlada_opo') == 'opozice')).select(pl.col('mluvci','id')).unique()

mluvci,id
str,str
"""Poslanec Ivan Bartoš""","""6433"""
"""Poslankyně Jana Berkovcová""","""6723"""
"""Poslankyně Iveta Štefanová""","""6785"""
"""Poslanec Lubomír Wenzl""","""6730"""
"""Poslanec David Kasal""","""6209"""
"""Poslanec Marek Novák""","""6533"""
"""Poslanec Roman Kubíček""","""6215"""
"""Poslanec Vít Rakušan""","""6524"""
"""Poslankyně Jana Mračková Vildumetzová""","""6471"""


In [313]:
df_vlady.filter((pl.col('datum') > pl.date(2022, 1, 1)) & (pl.col('vlada_opo') == 'opozice')).select(pl.col('mluvci','id')).unique()

mluvci,id
str,str
"""Poslanec Jan Richter""","""6500"""
"""Poslanec Jiří Kobza""","""6503"""
"""Místopředsedkyně PSP Olga Richterová""","""6473"""
"""Poslanec Jaroslav Bžoch""","""6441"""
"""Poslanec Bohuslav Hudec""","""6964"""
"""Poslanec Patrik Nacher""","""6487"""
"""Poslankyně Karla Maříková""","""6469"""
"""Poslankyně Klára Kocmanová""","""6776"""
"""Poslankyně Kamila Bláhová""","""6940"""


In [314]:
vyvoj = df_vlady.filter(
    pl.col('vlada_opo') == 'vláda'
).group_by_dynamic(
    index_column='datum',every='1mo'
).agg(
    pl.col('pocet_slov').sum()
).rename(
    {'pocet_slov':'slova_vlada'}
).join(df_vlady.filter(
    pl.col('vlada_opo') == 'opozice'
).group_by_dynamic(
    index_column='datum',every='1mo'
).agg(
    pl.col('pocet_slov').sum()
).rename(
    {'pocet_slov':'slova_opozice'}
), how='left', on='datum'
).with_columns(
    (pl.col('slova_vlada') / pl.col('slova_opozice')).alias('pomer')
)

In [315]:
alt.Chart(
    vyvoj.to_pandas()
).mark_line(
).encode(
    alt.X('datum:T'),
    alt.Y('pomer:Q')
)

alt.Chart(...)

In [316]:
def kontrola(jmeno):
    return df_vlady.filter(pl.col('mluvci').str.contains(jmeno)).group_by(['obdobi','vlada_opo']).agg(pl.col('text').unique().len()).sort(by='obdobi')

In [317]:
kontrola('Tomio')

obdobi,vlada_opo,text
i32,str,u32
2013,null,4
2013,"""opozice""",488
2017,null,55
2017,"""opozice""",7871
2021,"""opozice""",1745


In [318]:
kontrola('Babiš')

obdobi,vlada_opo,text
i32,str,u32
2013,"""vláda""",855
2017,"""vláda""",865
2021,"""vláda""",6
2021,"""opozice""",1535


In [319]:
kontrola('Petr Fiala')

obdobi,vlada_opo,text
i32,str,u32
2013,"""opozice""",345
2013,null,16
2017,"""opozice""",7124
2021,"""opozice""",5
2021,"""vláda""",707
2021,null,1


In [320]:
kontrola('Kateřina Konečná')

obdobi,vlada_opo,text
i32,str,u32
2013,null,1
2013,"""opozice""",58


In [321]:
kontrola('Marek Benda')

obdobi,vlada_opo,text
i32,str,u32
2013,"""opozice""",442
2013,null,5
2017,"""opozice""",566
2021,"""opozice""",4
2021,"""vláda""",434


In [322]:
kontrola('Ivan Bartoš')

obdobi,vlada_opo,text
i32,str,u32
2017,"""opozice""",412
2017,null,2
2021,"""vláda""",379
2021,"""opozice""",84
2021,null,44


In [323]:
kontrola('Schillerová')

obdobi,vlada_opo,text
i32,str,u32
2017,"""vláda""",977
2021,"""opozice""",1613
2021,"""vláda""",33


In [324]:
df_vlady.filter(pl.col("mluvci").str.contains("Ivan Bartoš")).filter(pl.col("vlada_opo").is_null())

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly,vlada,vlada_od,vlada_do,koalice_kandidatka,koalice_navrhovatel,opozice_kandidatka,opozice_navrhovatel,vlada_opo
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str],str,date,date,str,str,str,str,str
"""Poslanec Ivan Bartoš""","""2017ps_stenprot_001schuz_s001043.htm""","""/sqw/detail.sqw?id=6433""","""Vážený pane předsedo, vážené dámy, vážení pánové, veřejnosti, teď jsme viděli volbu, která skutečně nenaplnila to, co jsme tady vlastně celou dobu sly…",11,"""ps""",2017,1,43,2017-11-24,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""1. schůze""",true,true,true,"""6433""",158,"""6433_8.html""","""PhDr. Ivan Bartoš, Ph.D.""","""Středočeský""","""m""",2017-10-21,2021-10-20,"""Piráti""",null,"""1980""","[{""člen"",""Výbor pro veřejnou správu a regionální rozvoj"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Výbor pro bezpečnost"",""28. 11. 2017"",""15. 12. 2017""}, … {""člen"",""Skupina ČR - Jamajka"",""12. 6. 2018"",""20. 10. 2021""}]",1980,null,"""Ivan Bartoš""","[""PhDr."", ""Ph.D.""]","""vláda Bohuslava Sobotky""",2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""","""""",null
"""Poslanec Ivan Bartoš""","""2017ps_stenprot_002schuz_s002006.htm""","""/sqw/detail.sqw?id=6433""","""Vážený pane předsedo, vážené kolegyně, vážení kolegové, tady už zaznělo od mých předřečníků i od pana Vondráčka, jak jemné nuance můžou být v interpre…",2,"""ps""",2017,2,6,2017-11-28,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""2. schůze""",true,true,true,"""6433""",238,"""6433_8.html""","""PhDr. Ivan Bartoš, Ph.D.""","""Středočeský""","""m""",2017-10-21,2021-10-20,"""Piráti""",null,"""1980""","[{""člen"",""Výbor pro veřejnou správu a regionální rozvoj"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Výbor pro bezpečnost"",""28. 11. 2017"",""15. 12. 2017""}, … {""člen"",""Skupina ČR - Jamajka"",""12. 6. 2018"",""20. 10. 2021""}]",1980,null,"""Ivan Bartoš""","[""PhDr."", ""Ph.D.""]","""vláda Bohuslava Sobotky""",2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""","""""",null
"""Poslanec Ivan Bartoš""","""2021ps_stenprot_118schuz_s118002.htm""","""/sqw/detail.sqw?o=9&id=6433""","""Ještě jsem stále předseda Pirátské strany. Děkuji.""",3,"""ps""",2021,118,2,2024-11-06,"""PČR, PS 2021-...""","""118. schůze""",true,true,true,null,6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""vláda Petra Fialy bez Pirátů""",2024-10-01,2025-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09""","""ANO2011*SPD""","""Piráti""",null
"""Poslanec Ivan Bartoš""","""2021ps_stenprot_118schuz_s118002.htm""","""/sqw/detail.sqw?o=9&id=6433""","""Volby vedení Pirátské strany jsou až v sobotu, kdy bude zvoleno nové vedení. Děkuji.""",5,"""ps""",2021,118,2,2024-11-06,"""PČR, PS 2021-...""","""118. schůze""",true,true,true,null,13,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""vláda Petra Fialy bez Pirátů""",2024-10-01,2025-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09""","""ANO2011*SPD""","""Piráti""",null
"""Poslanec Ivan Bartoš""","""2021ps_stenprot_118schuz_s118002.htm""","""/sqw/detail.sqw?o=9&id=6433""","""Vážená paní předsedající, vážení páni ministři, vážené poslankyně, vážení poslanci, my jsme se dnes sešli kvůli takzvanému bypassu digitalizace staveb…",7,"""ps""",2021,118,2,2024-11-06,"""PČR, PS 2021-...""","""118. schůze""",true,true,true,null,897,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""vláda Petra Fialy bez Pirátů""",2024-10-01,2025-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09""","""ANO2011*SPD""","""Piráti""",null
""

In [325]:
df_vlady.filter(pl.col('datum').dt.year() == 2021).filter(pl.col('datum').dt.month() == 11).filter(pl.col('mluvci').str.contains('Stanjura'))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly,vlada,vlada_od,vlada_do,koalice_kandidatka,koalice_navrhovatel,opozice_kandidatka,opozice_navrhovatel,vlada_opo
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str],str,date,date,str,str,str,str,str
"""Poslanec Zbyněk Stanjura""","""2021ps_stenprot_001schuz_s001006.htm""","""/sqw/detail.sqw?id=5969""","""Děkuji za slovo. My jsme se dohodli s ostatními předsedy poslaneckých klubů, že počet členů mandátového a imunitního výboru Poslanecká sněmovna stanov…",3,"""ps""",2021,1,6,2021-11-08,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,"""5969""",27,"""5969_9.html""","""Ing. Zbyněk Stanjura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPOLU""","""ODS""","""15. 2. 1964""","[{""člen"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",null}, {""předsed"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",""13. 12. 2021""}, … {""člen"",""Dozorčí rada Státního fondu dopravní infrastruktury"",""15. 5. 2018"",""6. 4. 2022""}]",1964,1964-02-15,"""Zbyněk Stanjura""","[""Ing.""]","""druhá vláda Andreje Babiše""",2018-06-27,2021-12-17,"""ČSSD*ANO2011""","""""","""TOP09*KDU-ČSL*STAN*Piráti*ODS*SPD*KSČM""","""""","""opozice"""
"""Poslanec Zbyněk Stanjura""","""2021ps_stenprot_001schuz_s001010.htm""","""/sqw/detail.sqw?id=5969""","""Myslím, že je to úplně jednoduché. Pan ministr má přednostní právo. (Předsedající: Má, ale ústní.) Takže může vystoupit jako první a nemusíme se o tom…",3,"""ps""",2021,1,10,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,"""5969""",41,"""5969_9.html""","""Ing. Zbyněk Stanjura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPOLU""","""ODS""","""15. 2. 1964""","[{""člen"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",null}, {""předsed"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",""13. 12. 2021""}, … {""člen"",""Dozorčí rada Státního fondu dopravní infrastruktury"",""15. 5. 2018"",""6. 4. 2022""}]",1964,1964-02-15,"""Zbyněk Stanjura""","[""Ing.""]","""druhá vláda Andreje Babiše""",2018-06-27,2021-12-17,"""ČSSD*ANO2011""","""""","""TOP09*KDU-ČSL*STAN*Piráti*ODS*SPD*KSČM""","""""","""opozice"""
"""Poslanec Zbyněk Stanjura""","""2021ps_stenprot_001schuz_s001017.htm""","""/sqw/detail.sqw?id=5969""","""Děkuji za slovo. Já k tomu bodu vystoupím někdy později. Jenom krátká poznámka k nové politické kultuře, budu se tomu pak věnovat podrobněji. My jsme …",3,"""ps""",2021,1,17,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,"""5969""",134,"""5969_9.html""","""Ing. Zbyněk Stanjura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPOLU""","""ODS""","""15. 2. 1964""","[{""člen"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",null}, {""předsed"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",""13. 12. 2021""}, … {""člen"",""Dozorčí rada Státního fondu dopravní infrastruktury"",""15. 5. 2018"",""6. 4. 2022""}]",1964,1964-02-15,"""Zbyněk Stanjura""","[""Ing.""]","""druhá vláda Andreje Babiše""",2018-06-27,2021-12-17,"""ČSSD*ANO2011""","""""","""TOP09*KDU-ČSL*STAN*Piráti*ODS*SPD*KSČM""","""""","""opozice"""
"""Poslanec Zbyněk Stanjura""","""2021ps_stenprot_001schuz_s001029.htm""","""/sqw/detail.sqw?id=5969""","""Děkuji za slovo. Určitě ode mě nemůžete čekat, že bych kritizoval to, že někdo vystupuje, to určitě ne. Věta o tom obědě je má z minulého volebního ob…",6,"""ps""",2021,1,29,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,"""5969""",300,"""5969_9.html""","""Ing. Zbyněk Stanjura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPOLU""","""ODS""","""15. 2. 1964"

In [326]:
df_vlady.filter(pl.col('datum').dt.year() == 2021).filter(pl.col('datum').dt.month() == 11).select(pl.col(['mluvci','vlada_opo','datum'])).unique()

mluvci,vlada_opo,datum
str,str,date
"""Poslanec Karel Rais""","""vláda""",2021-11-10
"""Předsedkyně PSP Markéta Pekarová Adamová""",null,2021-11-23
"""Poslanec Pavel Žáček""","""opozice""",2021-11-10
"""Poslanec Marek Výborný""","""opozice""",2021-11-23
"""Poslanec Zbyněk Stanjura""","""opozice""",2021-11-08
"""Poslanec Radim Holiš""","""vláda""",2021-11-10
"""Poslanec Zbyněk Stanjura""","""opozice""",2021-11-10
"""Poslanec Michal Kučera""","""opozice""",2021-11-10
"""Poslanec Jan Volný""","""vláda""",2021-11-10


In [327]:
df_vlady.filter(pl.col('vlada_opo') == 'vláda').sample(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly,vlada,vlada_od,vlada_do,koalice_kandidatka,koalice_navrhovatel,opozice_kandidatka,opozice_navrhovatel,vlada_opo
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str],str,date,date,str,str,str,str,str
"""Ministr zemědělství ČR Marek Výborný""","""2021ps_stenprot_094schuz_s094134.htm""","""https://www.vlada.cz/cz/clenove-vlady/marek-vyborny-206784/""","""Vážená paní místopředsedkyně, kolegové z vlády, kolegyně, poslankyně a poslanci, děkuji za slovo. Cítím se povinován vystoupit, protože jsem pečlivě p…",5,"""ps""",2021,94,134,2024-03-05,"""PČR, PS 2021-...""","""94. schůze""",true,true,true,null,778,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""vláda Petra Fialy s Piráty""",2021-12-17,2024-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09*Piráti""","""ANO2011*SPD""","""""","""vláda"""
"""Poslanec Jan Chvojka""","""2013ps_stenprot_017schuz_s017335.htm""","""/sqw/detail.sqw?id=5926""","""Dále máme na řadě pozměňovací návrh můj. Samozřejmě moje stanovisko je kladné.""",7,"""ps""",2013,17,335,2014-09-26,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""17. schůze""",true,true,true,"""5926""",11,"""5926_7.html""","""JUDr. Jan Chvojka""","""Pardubický""","""m""",2013-10-26,2017-10-26,"""ČSSD""",null,"""1980""","[{""člen"",""Mandátový a imunitní výbor"",""27. 11. 2013"",""30. 11. 2016""}, {""člen"",""Volební výbor"",""4. 12. 2013"",""30. 11. 2016""}, … {null,""Vláda České republiky"",""1. 12. 2016"",""13. 12. 2017""}]",1980,null,"""Jan Chvojka""","[""JUDr.""]","""vláda Bohuslava Sobotky""",2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""","""""","""vláda"""
"""Poslanec Jiří Havránek""","""2021ps_stenprot_077schuz_s077111.htm""","""/sqw/detail.sqw?id=6755""","""Budeme tedy opakovat hlasování k pozměňujícímu návrhu C6, sněmovní dokument 3117, který se týká zákona o veřejných výzkumných institucích. Stanovisko …",18,"""ps""",2021,77,111,2023-10-13,"""PČR, PS 2021-...""","""77. schůze""",true,true,true,"""6755""",25,"""6755_9.html""","""Ing. Jiří Havránek""","""Středočeský""","""m""",2021-10-09,null,"""SPOLU""","""ODS""","""23. 12. 1992""","[{""člen"",""Výbor pro veřejnou správu a regionální rozvoj"",""23. 11. 2021"",null}, {""člen"",""Rozpočtový výbor"",""23. 11. 2021"",null}, … {""člen"",""Skupina ČR - Velká Británie"",""2. 3. 2022"",null}]",1992,1992-12-23,"""Jiří Havránek""","[""Ing.""]","""vláda Petra Fialy s Piráty""",2021-12-17,2024-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09*Piráti""","""ANO2011*SPD""","""""","""vláda"""
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_025schuz_s025127.htm""","""/sqw/detail.sqw?id=6178""","""Děkuji. Nyní s faktickou poznámkou paní Berenika Peštová. Prosím, vaše dvě minuty.""",8,"""ps""",2021,25,127,2022-06-03,"""PČR, PS 2021-...""","""25. schůze""",true,true,true,"""6178""",11,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Thajské království"",""4. 4. 2025"",null}]",1971,1971-11-10,"""Jan Bartošek""","[""Ing.""]","""vláda Petra Fialy s Piráty""",2021-12-17,2024-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09*Piráti""","""ANO2011*SPD""","""""","""vláda"""
"""Místopředseda PSP Jan Bartošek""","""2013ps_stenprot_007schuz_s007075.htm""","""/sqw/detail.sqw?id=6178""","""Děkuji panu zpravodaji. S faktickou poznámkou je přihlášen pan poslanec Marek Černoch. Máte slovo.""",4,"""ps""",2013,7,75,2014-03-19,"""Parlament Če

In [328]:
ministri = df.filter(pl.col("mluvci_id").str.contains("vlada.cz")).group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum())

In [329]:
alt.Chart(
    ministri.to_pandas()
).mark_line(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

## Muži x ženy

In [331]:
df.group_by(
    ["obdobi","gender"]
).agg(
    pl.col('pocet_slov').sum()
).pivot(
    index='obdobi',on='gender'
).with_columns(
    (pl.col('f') / ((pl.col('f')) + pl.col('m'))).alias('podil_zen')
).sort(
    by='obdobi'
)

obdobi,f,null,m,podil_zen
i32,u32,u32,u32,f64
1993,null,3681289,null,null
1996,null,4029334,161,null
1998,937,8955418,7028,0.11764
2002,1511422,594619,7291948,0.171687
2006,1012935,448013,5771503,0.149303
2010,842776,876624,4845730,0.148154
2013,1339930,2329520,7558184,0.150586
2017,1741753,2543637,8784252,0.165471
2021,3746958,2345743,9496322,0.282933


In [332]:
df.with_columns(
    pl.when(
        pl.col('mluvci').str.contains(r'ová[^\w]') | pl.col('mluvci').str.ends_with('á') | pl.col('mluvci').str.contains('oslankyně') | pl.col("mluvci").str.contains("sedkyně ")
    ).then(pl.lit('f')).otherwise(pl.lit('m')).alias('gender_naivni')
).group_by(
    ["obdobi","gender_naivni"]
).agg(
    pl.col('pocet_slov').sum()
).pivot(
    index='obdobi',on='gender_naivni'
).with_columns(
    (pl.col('f') / ((pl.col('f')) + pl.col('m'))).alias('podil_zen')
).sort(
    by='obdobi'
)

obdobi,m,f,podil_zen
i32,u32,u32,f64
1993,3453185,228104,0.061963
1996,3578468,451027,0.111931
1998,7827043,1136340,0.126776
2002,7826532,1571457,0.167212
2006,6162128,1070323,0.147989
2010,5193184,1371946,0.208975
2013,9652978,1574656,0.140248
2017,10686510,2383132,0.182341
2021,11385561,4203462,0.269642


In [333]:
df.filter(
    (pl.col('mluvci').str.contains(r'ová[^\w]') | pl.col('mluvci').str.ends_with('á') | pl.col('mluvci').str.contains('oslankyně') | pl.col("mluvci").str.contains("sedkyně "))
).group_by("mluvci").agg(pl.col("text").unique().len()).sort(by="text",descending=True)

mluvci,text
str,u32
"""Místopředsedkyně PSP Petra Buzková""",12857
"""Místopředsedkyně PSP Miroslava Němcová""",9448
"""Předsedkyně PSP Miroslava Němcová""",7985
"""Místopředsedkyně PSP Jitka Kupčová""",6187
"""Místopředsedkyně PSP Olga Richterová""",5888
"""Místopředsedkyně PSP Věra Kovářová""",5588
"""Místopředsedkyně PSP Jaroslava Jermanová""",4795
"""Předsedkyně PSP Markéta Pekarová Adamová""",4782
"""Místopředsedkyně PSP Lucie Talmanová""",4189


In [334]:
df.filter(
    ~(pl.col('mluvci').str.contains(r'ová[^\w]') | pl.col('mluvci').str.ends_with('á') | pl.col('mluvci').str.contains('oslankyně') | pl.col("mluvci").str.contains("sedkyně "))
).group_by("mluvci").agg(pl.col("text").unique().len()).sort(by="text",descending=True)

mluvci,text
str,u32
"""Místopředseda PSP Vojtěch Filip""",28877
"""Místopředseda PSP Jan Kasal""",16449
"""Místopředseda PSP Ivan Langer""",15441
"""Místopředseda PSP Jan Bartošek""",14214
"""Místopředseda PSP Lubomír Zaorálek""",10394
"""Místopředseda PSP Petr Gazdík""",9199
"""Předseda PSP Lubomír Zaorálek""",8528
"""Místopředseda PSP Tomio Okamura""",7924
"""Místopředseda PSP Vojtěch Pikal""",7496


In [335]:
df.filter(pl.col("gender") == "f").filter(
    ~(pl.col('mluvci').str.contains(r'ová[^\w]') | pl.col('mluvci').str.ends_with('á') | pl.col('mluvci').str.contains('oslankyně') | pl.col("mluvci").str.contains("sedkyně "))
).group_by("mluvci").agg(pl.col("text").unique().len()).sort(by="text",descending=True)

mluvci,text
str,u32
"""Náměstek primátora hl. m. Prahy Rudolf Blažek""",22
"""Guvernér ČNB Zdeněk Tůma""",17
"""Senátor Jaroslav Kubera""",14
"""Senátor Martin Mejstřík""",11
"""Senátor Josef Novotný""",6
"""Pan Rudolf Blažek""",6
"""Senátor Luděk Sefzig""",6
"""Hejtman Ústeckého kraje Jiří Šulc""",6
"""Senátor Adolf Jílek""",5


In [336]:
df.filter(pl.col('mluvci') == 'Náměstek primátora hl. m. Prahy Rudolf Blažek')

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Náměstek primátora hl. m. Prahy Rudolf Blažek""","""2002ps_stenprot_016schuz_s016023.htm""","""/sqw/detail.sqw?id=64""","""Děkuji, hezký dobrý den. Vážený pane předsedající, vážené dámy, vážení pánové, podruhé před vás předstupuji s návrhem hlavního města Prahy na schválen…",3,"""ps""",2002,16,23,2003-05-13,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""16. schůze""",true,true,true,"""64""",191,"""64_4.html""","""Ing. Jan Kasal""","""Vysočina""","""m""",2002-06-15,2006-06-15,"""K""","""KDU-ČSL""","""1951""","[{""člen"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""místopředsed"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""předsed"",""Skupina ČR - Rusko a Eurasie"",""6. 3. 2003"",""10. 2. 2005""}]",1951,null,"""Jan Kasal""","[""Ing.""]"
"""Náměstek primátora hl. m. Prahy Rudolf Blažek""","""2002ps_stenprot_022schuz_s022009.htm""","""/sqw/detail.sqw?id=252""","""Vážený pane předsedo, vážené paní poslankyně, vážení páni poslanci, děkuji za udělení slova. K našemu návrhu bych chtěl uvést, že dnes před vás již po…",7,"""ps""",2002,22,9,2003-10-14,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""22. schůze""",true,true,true,"""252""",172,"""252_4.html""","""PhDr. Lubomír Zaorálek""","""Moravskoslezský""","""m""",2002-06-15,2006-06-15,"""ČSSD""",null,"""1956""","[{""člen"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""předsed"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""21. 6. 2002"",""15. 6. 2006""}]",1956,null,"""Lubomír Zaorálek""","[""PhDr.""]"
"""Náměstek primátora hl. m. Prahy Rudolf Blažek""","""2002ps_stenprot_030schuz_s030161.htm""","""/sqw/detail.sqw?id=401""","""Vážená paní místopředsedkyně, vážené paní poslankyně, vážení poslanci, předkládám vám k posouzení návrh hl. m. Prahy na vydání zákona, kterým se mění …",7,"""ps""",2002,30,161,2004-03-30,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""30. schůze""",true,true,true,"""401""",761,"""401_4.html""","""Miroslava Němcová""","""Vysočina""","""f""",2002-06-15,2006-06-15,"""ODS""",null,"""1952""","[{""člen"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""místopředsed"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Skupina ČR - Izrael"",""11. 12. 2002"",""15. 6. 2006""}]",1952,null,"""Miroslava Němcová""",[]
"""Náměstek primátora hl. m. Prahy Rudolf Blažek""","""2002ps_stenprot_030schuz_s030163.htm""","""/sqw/detail.sqw?id=401""","""Děkuji, vážená paní předsedající. Dámy a pánové, v zastoupení hlavního města Prahy předkládám dnes druhý návrh zákona, a to návrh zákona, kterým se mě…",2,"""ps""",2002,30,163,2004-03-30,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""30. schůze""",true,true,true,"""401""",1212,"""401_4.html""","""Miroslava Němcová""","""Vysočina""","""f""",2002-06-15,2006-06-15,"""ODS""",null,"""1952""","[{""člen"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""místopředsed"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Skupina ČR - Izrael"",""11. 12. 2002"",""15. 6. 2006""}]",1952,null,"""Miroslava Němcová""",[]
"""Náměstek primátora hl. m. Prahy Rudolf Blažek""","""2002ps_stenprot_030schuz_s030164.htm""","""/sqw/detail.sqw?id=401""","""(17.40 hodin) Tento zásah uvedený v návrhu zákona by byl prováděn na základě zákona ve veřejném zájmu a za náhradu, takže jsou dodrženy všechny ústav…",1,"""ps""",2002,30,164,2004-03-30,"""Parlament České republiky, Pos